############# Markdown note ##################

<div class="alert alert-block alert-info"> <b>NOTE</b> Use blue boxes for Tips and notes. </div>

<div class="alert alert-block alert-success"> Use green boxes sparingly, and only for some specific purpose that the other boxes can't cover. For example, if you have a lot of related content to link to, maybe you decide to use green boxes for related links from each section of a notebook. </div>

<div class="alert alert-block alert-warning"> Use yellow boxes for examples that are not inside code cells, or use for mathematical formulas if needed. </div>

<div class="alert alert-block alert-danger"> In general, just avoid the red boxes. </div>

<img src="<path>" width=20% style="margin-left:auto; margin-right:auto">
<img src="<path>" width=40% style="float: right;">  

In [ ]:
%%sh

# reset all programs
rm -rf debug*

# MPI Introduction

An introduction to basic concept of **Message Passing Interface** (MPI)


<div class="alert alert-block alert-warning"> The MPI is a <b>standard</b> interface ruled by <a href="https://www.mpi-forum.org/">MPI Forum</a>. </div>

Born in 1992, first version released in 1994.

Versions:
* *MPI-1*, 1994
* *MPI-2*, 1996
* *MPI-3*, 2014
* *MPI-4*, 2021

<div class="alert alert-block alert-danger">MPI for C++ is deprecated from version MPI 3.0</div>

<div class="alert alert-block alert-warning"> Different <b>implementation</b> of the standard: </div>

* C/C++;
* Fortran;
* Pyhton;
* ...

Both **open source** and **proprietary**:
* [open-mpi](https://www.open-mpi.org/)
* [mpich](https://www.mpich.org/)
* [intelmpi](https://www.intel.com/content/www/us/en/developer/tools/oneapi/mpi-library.html) (proprietary)
* [mpi4py](https://mpi4py.readthedocs.io/en/stable/index.html)

<div class="alert alert-block alert-info"> <b>DOCUMENTATION</b>: we are going to use C/C++ <a href="https://www.open-mpi.org/doc/v4.1/">https://www.open-mpi.org/doc/v4.1/</a>.</div>

<div class="alert alert-block alert-success"> For some implementations (e.g. openmpi, intelmpi) multiple <b>compilers</b> are available. </div>

## MPI - Start coding

### To start

* For C++ we are going to use **mpic++** compiler (extension of **g++**) with `openmpi` MPI implementation.
* For Python we use the `mpi4py` library.


#### C++ (1)

C++ Source file (*.cpp) + CMake file (CMakeLists.txt)

In [ ]:
%%writefile main_example.cpp

#include <iostream> // output library 
#include <mpi.h> // MPI library

int main(int argc, char **argv) 
{
    std::cout<< "Hello world!"<< std::endl;
    return 0;
}

In [ ]:
%%writefile CMakeLists.txt

cmake_minimum_required(VERSION 3.2)
project(1_MPI_Basic LANGUAGES CXX C VERSION 1.0.0) # Name of the project

set(SOURCES "" CACHE STRING "The sources list") # Variable which stores the cpp files

find_package(MPI REQUIRED) # Find the MPI library

add_executable(${PROJECT_NAME} ${SOURCES}) # add list of files to executable
target_link_libraries(${PROJECT_NAME} MPI::MPI_CXX) # Link MPI library

#### C++ (2)

**Compile** and run code from terminal:

In [ ]:
%%sh
# compile program
mkdir -p ./debug_example
cd debug_example
cmake -DSOURCES="main_example.cpp" ..
make

In [ ]:
%%sh
# run program serial
cd debug_example
./1_MPI_Basic # classic call
mpirun -np 1 1_MPI_Basic # mpi call

In [ ]:
%%sh
# run program in parallel
cd debug_example
mpirun -np 2 1_MPI_Basic

In [ ]:
%%sh
# run program with more processes
cd debug_example
mpirun --oversubscribe -np 10 1_MPI_Basic

#### Python

Python Source file (*.py) and run code from terminal (**No Compilation step**)

In [ ]:
%%writefile main_example.py

import mpi4py # import mpi4py module
mpi4py.rc.initialize = False  # do not initialize MPI automatically
mpi4py.rc.finalize = False    # do not finalize MPI automatically

from mpi4py import MPI # import the 'MPI' module

print("Hello world!")

In [ ]:
%%sh
# run program serial
python main_example.py # classic run
mpirun -np 1 python main_example.py # mpi serial run

In [ ]:
%%sh
# run program in parallel
mpirun -np 2 python main_example.py

In [ ]:
%%sh
# run program with more processes
mpirun --oversubscribe -np 10 python main_example.py

## MPI Parallel session - Init and Finalize

### Init and Finalize

Calls used to initialize and terminate the parallel session.

* `MPI_Init`: see https://www.open-mpi.org/doc/v4.1/man3/MPI_Init.3.php
* `MPI_Finalize`: see https://www.open-mpi.org/doc/v4.1/man3/MPI_Finalize.3.php

### C++ (1)

In [ ]:
%%writefile main_init.cpp

#include <iostream>
#include <mpi.h>

int main(int argc, char **argv) 
{
    // Initialize MPI
    // This must always be called before any other MPI functions
    MPI_Init(&argc, &argv);
    
    std::cout<< "Hello world!"<< std::endl;

    // Finalize MPI
    // This must always be called after all other MPI functions
    MPI_Finalize();

    return 0;
}

<div class="alert alert-block alert-info"> The arguments in <code>MPI_Init</code> are <strong>not used</strong> anymore but some compilers insist they are there. </div>

### C++ (2)

In [ ]:
%%sh

# compile program
mkdir -p ./debug_init
cd debug_init
cmake -DSOURCES="main_init.cpp" ..
make

In [ ]:
%%sh

# run program
cd debug_init
mpirun -np 4 1_MPI_Basic

### Python

In [ ]:
%%writefile main_init.py

import mpi4py
mpi4py.rc.initialize = False  # do not initialize MPI automatically
mpi4py.rc.finalize = False    # do not finalize MPI automatically

from mpi4py import MPI # import the 'MPI' module

# manual initialization of the MPI environment
MPI.Init()

print("Hello world!")

# manual finalization of the MPI environment
MPI.Finalize()

In [ ]:
%%sh

# run program
mpirun -np 4 python main_init.py

## MPI Communicators

It is possible to divide the total number of tasks into groups called **communicators**.
The variable identifying a communicator identifies those tasks which can communicate with each other.

<div class="alert alert-block alert-info">The default communicator is called <code>MPI_COMM_WORLD</code> and includes <b>all</b> the tasks available to the program.</div>

<img src="./Images/COMM_WORLD.png" width=20% style="margin-left:auto; margin-right:auto">


* `MPI_Comm_size`: see https://www.open-mpi.org/doc/v4.1/man3/MPI_Comm_size.3.php
* `MPI_Comm_rank`: see https://www.open-mpi.org/doc/v4.1/man3/MPI_Comm_rank.3.php

In [ ]:
%%writefile main_communicators.cpp

#include <iostream>
#include <mpi.h>

int main(int argc, char **argv)
{
    int err;
    err = MPI_Init(&argc, &argv);
    
    int nprocs, my_rank;
    
    // Get the number of processes in MPI_COMM_WORLD
    err = MPI_Comm_size(MPI_COMM_WORLD, &nprocs);
    
    // Get the rank of this process in MPI_COMM_WORLD
    err = MPI_Comm_rank(MPI_COMM_WORLD, &my_rank);

    std::cout<< "Hello I am process "<< my_rank<< " of "<< nprocs<< " processes"<< std::endl; 
    
    err = MPI_Finalize();
    
    return 0;
}

<div class="alert alert-block alert-success">
    <ol>
        <li> Remember that <b>every</b> process is running the same code independently 
        <li> At the end of the call, rank will have a <b>different</b> value for every process!
    </ol>
</div>

In [ ]:
%%sh

# compile program
mkdir -p ./debug_communicators
cd debug_communicators
cmake -DSOURCES="main_communicators.cpp" ..
make

In [ ]:
%%sh

# run program
cd debug_communicators
mpirun -np 4 1_MPI_Basic

Communicators can be created with different MPI routines:
* `MPI_Comm_split`: https://www.open-mpi.org/doc/v4.1/man3/MPI_Comm_split.3.php
* `MPI_Comm_dup`: https://www.open-mpi.org/doc/v4.1/man3/MPI_Comm_dup.3.php
* `MPI_Comm_create`: https://www.open-mpi.org/doc/v4.1/man3/MPI_Comm_create.3.php
* ...

<div class="alert alert-block alert-danger"> <b>REMARK</b>: Once a new communicator is created we shall <b>destroy</b> it before closing the application:</div>

* `MPI_Comm_free`: https://www.open-mpi.org/doc/v4.1/man3/MPI_Comm_free.3.php

<div class="alert alert-block alert-info"> <b>NOTE</b>: We are going to see more about communicators when we are going to see <b>Topologies</b>. </div>


### Example of MPI Comm Split

`MPI_Comm_split`: https://www.open-mpi.org/doc/v4.1/man3/MPI_Comm_split.3.php
<img src="./Images/comm_split.png" width=40% style="margin-left:auto; margin-right:auto">

In [ ]:
%%writefile main_comm_split.cpp

#include <iostream>
#include <mpi.h>
int main(int argc, char **argv)
{
    int err;
    MPI_Init(&argc, &argv);
    
    int nprocs, my_rank;
    
    // Get the rank and size in the original communicator
    int world_rank, world_size;
    MPI_Comm_rank(MPI_COMM_WORLD, &world_rank);
    MPI_Comm_size(MPI_COMM_WORLD, &world_size);

    unsigned int color = world_rank / 4; // Determine color based on row
    
    // Split the communicator based on the color and use the
    // original rank for ordering
    MPI_Comm row_comm;
    err = MPI_Comm_split(MPI_COMM_WORLD, color, world_rank, &row_comm);
    
    int row_rank, row_size;
    MPI_Comm_rank(row_comm, &row_rank);
    MPI_Comm_size(row_comm, &row_size);
    
    std::cout<< "WORLD RANK/SIZE: "<< world_rank<< "/"<< world_size<< "\t";
    std::cout<< "ROW RANK/SIZE: "<< row_rank<< "/"<< row_size<< std::endl;
    
    // Free the communicator
    MPI_Comm_free(&row_comm);
    
    MPI_Finalize();
    
    return 0;
}

In [ ]:
%%sh

# compile program
mkdir -p ./debug_comm_split
cd debug_comm_split
cmake -DSOURCES="main_comm_split.cpp" ..
make

In [ ]:
%%sh

# run program
cd debug_comm_split
mpirun --oversubscribe -np 16 1_MPI_Basic